In [1]:
import numpy as np
protarr = np.load("inputs/"+"5LOS_A_pt.npy")

In [2]:
print(protarr.shape) # [L,1024] -> [118, 1024]

(118, 1024)


In [3]:
# Importing necessary libraries
import time
import re
import torch
import argparse
import numpy as np
from tqdm import tqdm
# from transformers import AutoModel, AutoTokenizer  # Changed for ProtBert
from transformers import BertModel, BertTokenizer
# from transformers import BertForMaskedLM, BertTokenizer, pipeline
from dataset.data_functions import read_list, read_fasta_file

In [4]:
file_list = "file_lists/file_list_all.txt"
device = 'cuda:0'
model_name = "Rostlab/prot_bert"
# model_name = "Rostlab/prot_bert_bfd"
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=False)
model = BertModel.from_pretrained(model_name)
# model = BertForMaskedLM.from_pretrained(model_name)

In [5]:
dev='cuda:0'
device = torch.device(dev)
model = model.to(dev)
model = model.eval()
prot_list = read_list(file_list)
print(prot_list[0])

/mnt/nvme/home/bbabatun/IDL/PROJECT/SPOT-1D-LM/spot_1d_lm/fasta/T0859-D1.fasta


In [6]:
prot_path = prot_list[0]
# Read the protein sequence from the file
seq = read_fasta_file(prot_path)
# print(seq)
# Extracting the protein name from the file path
prot_name = prot_path.split('/')[-1].split('.')[0]
# print(prot_name)

# Insert spaces between each character in the sequence
seq_temp = seq.replace('', " ")
# print(seq_temp)
# Create a list of sequences (in this case, only one)
sequences_Example = [seq_temp]
# print(sequences_Example)
# Replace certain characters with 'X' for normalization
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]
# print(sequences_Example)
# Tokenize the sequences
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True)
# ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding="longest")

# Convert the tokenized sequences to PyTorch tensors and move to the specified device
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [7]:
print("Input IDs shape:", input_ids.shape)
print("Attention Mask shape:", attention_mask.shape)
print("Input IDs type:", input_ids.dtype)
print("Attention Mask type:", attention_mask.dtype)

Input IDs shape: torch.Size([1, 135])
Attention Mask shape: torch.Size([1, 135])
Input IDs type: torch.int64
Attention Mask type: torch.int64


In [8]:

# Generate embeddings with no gradient calculations (for efficiency)
with torch.no_grad():
    embedding = model(input_ids=input_ids, attention_mask=attention_mask)

# Move embeddings to CPU if using GPU and convert to numpy array
if dev == "cpu":
    embedding = embedding.last_hidden_state.numpy()
else:
    embedding = embedding.last_hidden_state.cpu().numpy()

# Extract and process embeddings for each sequence
features = []
for seq_num in range(len(embedding)):
    # Calculate the actual sequence length (excluding padding)
    seq_len = (attention_mask[seq_num] == 1).sum()
    # Extract the embeddings for the actual sequence length
    seq_emd = embedding[seq_num][:seq_len - 1]
    features.append(seq_emd)

In [9]:
features[0].shape

(134, 1024)